# TDDFT with Psi4

Roberto Di Remigio ([@robertodr](https://github.com/robertodr))

[European National Competence Centre Sweden](https://enccs.se/)

## At a glance

## Using Psithon

The bridge to Psithon was built by Jeff Schriber ([@jeffschriber](https://github.com/jeffschriber)) and Lori Burns ([@loriab](https://github.com/loriab))

- How does the input file look?
- How does the output file look?

## Using PsiAPI

- How can I write a script to run TDDFT?
- What variables are defined after a TDDFT run?

In [16]:
import psi4

from psi4.driver.procrouting.response.scf_response import tdscf_excitations

psi4.core.set_output_file("moxy.out")
psi4.core.set_num_threads(4)

moxy = psi4.geometry("""0 1
C  0.152133 -0.035800  0.485797
C -1.039475  0.615938 -0.061249
C  1.507144  0.097806 -0.148460
O -0.828215 -0.788248 -0.239431
H  0.153725 -0.249258  1.552136
H -1.863178  0.881921  0.593333
H -0.949807  1.214210 -0.962771
H  2.076806 -0.826189 -0.036671
H  2.074465  0.901788  0.325106
H  1.414895  0.315852 -1.212218
""", name="(S)-methyloxirane")

psi4.set_options({
    "save_jk": True,
})

e, wfn = psi4.energy("HF/cc-pVDZ", return_wfn=True, molecule=moxy)
res = tdscf_excitations(wfn, states=10)

## Known limitations

The implementation cannot currently handle the following cases: 
- The use of symmetry with density-fitted two-electron integrals. 
- Excited states of triplet symmetry from a restricted DFT reference. 
- Excited states from an unrestricted reference other than HF or LDA.

## Theory and implementation

For the Tamm-Dancoff approximation (TDA), the code uses a Davidson solver implemented by Ruhee D'Cunha ([@rdcunha](https://github.com/rdcunha)). For the random-phase approximation (RPA), the code uses a Hamiltonian solver implemented by Andrew James ([@amjames](https://github.com/amjames)). 
Both solvers are coded in Python and use the C++ implementation of the electronic Hessian-trial vector products implemented by Daniel Smith ([@dgasmith](https://https://github.com/dgasmith)).

## Plotting spectra

The length-gauge rotatory strengths Psi4 computes are currently **not** gauge-origin invariant.

$$
\varepsilon(\omega) = 
      \frac{4\pi^{2}N_{\mathrm{A}}\omega}{3\times 1000\times \ln(10) (4 \pi \epsilon_{0}) n \hbar c}
      \sum_{i \rightarrow j}g_{ij}(\omega)|\mathbf{\mu}_{ij}|^{2}
$$

$$
\Delta\varepsilon(\omega) =
      \frac{16\pi^{2}N_{\mathrm{A}}\omega}{3\times 1000\times \ln(10) (4 \pi \epsilon_{0}) n \hbar c^{2}}
      \sum_{i \rightarrow j}g_{ij}(\omega)\Im(\mathbf{\mu}_{ij}\cdot\mathbf{m}_{ij})
$$

See [Rizzo2011-to](https://doi.org/10.1002/9781118008720.ch2)

I decided to decouple the lineshape fitting from the plotting:
- We do not entangle Psi4 with any specific plotting library.
- The user has the freedom to run whatever visualization pipeline.

Psi4 provides the `spectrum` function:

```python
def spectrum(*,
             poles: Union[List[float], np.ndarray],
             residues: Union[List[float], np.ndarray],
             kind: str = "opa",
             lineshape: str = "gaussian",
             gamma: float = 0.2,
             npoints: int = 5000,
             out_units: str = "nm") -> Dict[str, np.ndarray]
```

that will perform the lineshape fitting of the TDDFT results and return numpy arrays for the $x$ and $y$ values of the plot.

For both **one-photon absorption** (OPA) and **electronic circular dichroism** (ECD) the *poles* are the excitation energies. The residues are:

- The square of the transition electric dipole moment for OPA: $|\mu_{ij}|^{2}$.
- The rotatory strength $R_{ij}$ for ECD. This is the imaginary part of the product of transition electric and magnetic dipole moments: $\Im(\mathbf{\mu}_{ij}\cdot\mathbf{m}_{ij})$.

The implementation assumes these quantities are given in the **length gauge**.

We can obtain these quantities in atomic units from the list of results returned by the `tdscf_excitations` function.

In [5]:
import numpy as np

# get poles and residues to plot OPA and ECD spectra
poles = [r["EXCITATION ENERGY"] for r in res]
opa_residues = [np.linalg.norm(r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"])**2 for r in res]
ecd_residues = [r["ROTATORY STRENGTH (LEN)"] for r in res]

With these at hand, we invoke the `spectrum` function to perform the convolution with Gaussian lineshapes (inhomogeneous broadening) with empirical linewidth $\gamma = 0.01\,\textrm{a.u.}$ (angular frequency). 

In [11]:
from psi4.driver.p4util import spectrum

opa_spectrum = spectrum(poles=poles, residues=opa_residues, gamma=0.01, out_units="nm")

ecd_spectrum = spectrum(poles=poles, residues=ecd_residues, kind="ECD", gamma=0.01, out_units="nm")

All data in input is given in atomic units, but the final spectrum will use nanometers on the $x$ axis and macroscopic units $\mathrm{L}\cdot\mathrm{mol}^{-1}\cdot\mathrm{cm}^{-1}$ on the $y$ axis.

The return value is a dictionary-of-dictionaries with:
1. The lineshape convolution:
   - `opa_spectrum["convolution"]["x"]` a NumPy array with the $x$ axis points of the lineshape convolution.
   - `opa_spectrum["convolution"]["y"]` a NumPy array with the $y$ axis points of the lineshape convolution.
2. The infinitely narrow (stick) representation:
   - `opa_spectrum["sticks"]["poles"]` a NumPy array with the poles in the chosen unit of measure.
   - `opa_spectrum["sticks"]["residues"]` a NumPy array with the residues in macroscopic units.

In [15]:
opa_spectrum["sticks"]

{'poles': array([124.07375252, 118.03023459, 115.43690877, 112.12611621,
        109.32211753, 109.08955351, 106.82893788, 104.23903833,
        103.73414661,  99.94543184]),
 'residues': array([3.10116093e+00, 1.07161409e+01, 2.08483761e+02, 1.02133229e+04,
        5.44012797e+03, 5.80576170e+03, 6.79281405e+03, 1.28870924e+03,
        1.42292033e+03, 5.19938683e+03])}

We can now plot these spectra with any library we want. In this example, I will use [Altair](https://altair-viz.github.io/) and [pandas](https://pandas.pydata.org/). The raw data is stored in NumPy arrays and any plotting library will work!

In [19]:
from altair_spectrum import plot_spectrum

opa_plot = plot_spectrum(opa_spectrum,
                         title="OPA (Gaussian broadening)",
                         x_title=("λ", "nm"))

ecd_plot = plot_spectrum(ecd_spectrum,
                         title="ECD (Gaussian broadening)",
                         x_title=("λ", "nm"),
                         y_title=("Δε", "L⋅mol⁻¹⋅cm⁻¹"))

opa_plot & ecd_plot

alt.VConcatChart(...)

## Solvent effects

Functionality provided by Holger Kruse ([@hokru](https://github.com/hokru)) and Maximilian Scheurer ([@maxscheurer](https://github.com/maxscheurer))